# Saving Models Insights:
## Model save and load functionality similar to Tensorflow.
* Save inputs .state_dict() for model and for optimizer - requires file path. model.save({dict of model and opt}, filepath)
* load


In [63]:
# Own implementation following structure from scratch.
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as f
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

In [64]:
# Forgot hyperparams, here:
batch_size = 32
input_size = 27648 # 96x96 for image
num_epochs = 15
learning_rate = 0.001
num_classes= 2

In [65]:
num_layers = [420, 69, 36, 27, 9, 3] # very special numbers indeed...
class IF_PCAN_ICAN_NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(IF_PCAN_ICAN_NN, self).__init__()
    self.input_layer = nn.Linear(input_size, num_layers[0])
    self.list_layers = [nn.Linear(num_layers[i], num_layers[i+1]) for i in range(len(num_layers)-1)]
    self.output = nn.Linear(num_layers[-1], num_classes)
  def forward(self, x):
    x = self.input_layer(x)
    for fu in self.list_layers:
      x = f.relu(fu(x))
    x = f.sigmoid(self.output(x))
    return x

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [67]:
import shutil
shutil.rmtree("PCAN-dst")

In [68]:
train_dst = datasets.PCAM('PCAN-dst/', split='test', transform=ToTensor(), download=True)
train_loader = DataLoader(train_dst, batch_size = batch_size, shuffle=True)
test_dst = datasets.PCAM('PCAN-dst/', split='val', transform=ToTensor(), download=True)
test_loader = DataLoader(test_dst, batch_size = batch_size, shuffle=True)

800875929it [00:08, 93857497.00it/s]
3040it [00:00, 28525020.49it/s]
805965320it [00:10, 79903696.76it/s] 
3038it [00:00, 4412152.20it/s]


In [69]:
model = IF_PCAN_ICAN_NN(input_size, num_classes).to(device)
rand_input = torch.rand(32, 27648)
model(rand_input).shape

torch.Size([32, 2])

In [70]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adamax(model.parameters(), lr=learning_rate)

In [77]:
def save_check_point(state, filename):
  torch.save(state, filename)
for i in range(1, num_epochs+1):
  if i % 5 == 0:
    checkpoint = {"state_dict": model.state_dict(),"optim": optimizer.state_dict()}
    save_check_point(checkpoint, f"{i}th_checkpoint.pth.tar")
  for (data, targets) in train_loader:
    x = data.to(device)
    y = targets.to(device)

    x = x.reshape(x.shape[0], -1)
    scores = model(x)
    _, predictions = scores.max(1)

    optimizer.zero_grad()
    losses = criterion(targets.float(), predictions.round().float())
    optimizer.step()

KeyboardInterrupt: ignored

In [79]:
#save model and load
s = torch.load('some_filepath')
#init weights in model
model.load_state_dict(s['state_dict']) # and with optimizer

FileNotFoundError: ignored

# Transfer Learning Insights
## Transfer Learning -> same concepts to Pytorch
1. Built in models, video -> torchvision.models, pretrained param
2. Disable trainable, i in model.parameters() -> i.requires_grad=False
3. Model layers accessed by .pool, .identity, etc. -> can set to new layer/model itself.
